In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.{col}

In [ ]:
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.feature.{VectorAssembler}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{GBTClassificationModel, GBTClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}

In [2]:
val spark = SparkSession
      .builder
      .master("local[1]")
      .getOrCreate

spark = org.apache.spark.sql.SparkSession@44e6b7da


org.apache.spark.sql.SparkSession@44e6b7da

In [3]:
val train_path_csv = "file:////share/slaba05data/lab05_train.csv"
val test_path_csv = "file:////share/slaba05data/lab05_test.csv"

train_path_csv = file:////share/slaba05data/lab05_train.csv
test_path_csv = file:////share/slaba05data/lab05_test.csv


file:////share/slaba05data/lab05_test.csv

# Train model

In [4]:
var trainDf = spark
    .read
    .format("csv")
    .option("header", "true")
    .option("delimiter", ",")
    .option("inferSchema", "true")
    .load(train_path_csv)
    .drop("_c0")
    .cache

trainDf.show(1, 0, true)

-RECORD 0--------------------------------------------
 ID                          | 479990                
 CR_PROD_CNT_IL              | 0                     
 AMOUNT_RUB_CLO_PRC          | 0.0                   
 PRC_ACCEPTS_A_EMAIL_LINK    | 0.0                   
 APP_REGISTR_RGN_CODE        | null                  
 PRC_ACCEPTS_A_POS           | 0.0                   
 PRC_ACCEPTS_A_TK            | 0.0                   
 TURNOVER_DYNAMIC_IL_1M      | 0.0                   
 CNT_TRAN_AUT_TENDENCY1M     | null                  
 SUM_TRAN_AUT_TENDENCY1M     | null                  
 AMOUNT_RUB_SUP_PRC          | 0.0                   
 PRC_ACCEPTS_A_AMOBILE       | 0.0                   
 SUM_TRAN_AUT_TENDENCY3M     | null                  
 CLNT_TRUST_RELATION         | null                  
 PRC_ACCEPTS_TK              | 0.0                   
 PRC_ACCEPTS_A_MTP           | 0.0                   
 REST_DYNAMIC_FDEP_1M        | 0.0                   
 CNT_TRAN_AUT_TENDENCY3M    

trainDf = [ID: int, CR_PROD_CNT_IL: int ... 114 more fields]


[ID: int, CR_PROD_CNT_IL: int ... 114 more fields]

In [5]:
var testDf = spark
    .read
    .format("csv")
    .option("header", "true")
    .option("delimiter", ",")
    .option("inferSchema", "true")
    .load(test_path_csv)
    .drop("_c0")
    .na.fill(0)
    .cache
trainDf.show(1, 0, true)

-RECORD 0--------------------------------------------
 ID                          | 479990                
 CR_PROD_CNT_IL              | 0                     
 AMOUNT_RUB_CLO_PRC          | 0.0                   
 PRC_ACCEPTS_A_EMAIL_LINK    | 0.0                   
 APP_REGISTR_RGN_CODE        | null                  
 PRC_ACCEPTS_A_POS           | 0.0                   
 PRC_ACCEPTS_A_TK            | 0.0                   
 TURNOVER_DYNAMIC_IL_1M      | 0.0                   
 CNT_TRAN_AUT_TENDENCY1M     | null                  
 SUM_TRAN_AUT_TENDENCY1M     | null                  
 AMOUNT_RUB_SUP_PRC          | 0.0                   
 PRC_ACCEPTS_A_AMOBILE       | 0.0                   
 SUM_TRAN_AUT_TENDENCY3M     | null                  
 CLNT_TRUST_RELATION         | null                  
 PRC_ACCEPTS_TK              | 0.0                   
 PRC_ACCEPTS_A_MTP           | 0.0                   
 REST_DYNAMIC_FDEP_1M        | 0.0                   
 CNT_TRAN_AUT_TENDENCY3M    

testDf = [ID: int, CR_PROD_CNT_IL: int ... 113 more fields]


[ID: int, CR_PROD_CNT_IL: int ... 113 more fields]

In [6]:
val categoriesCols = List(
    "CLNT_TRUST_RELATION", 
    "APP_MARITAL_STATUS",
    "APP_KIND_OF_PROP_HABITATION", 
    "CLNT_JOB_POSITION_TYPE",
    "CLNT_JOB_POSITION", 
    "APP_DRIVING_LICENSE", 
    "APP_EDUCATION",
    "APP_TRAVEL_PASS", 
    "APP_CAR", 
    "APP_POSITION_TYPE", 
    "APP_EMP_TYPE",
    "APP_COMP_TYPE", 
    "PACK"
)

categoriesCols = List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)


List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)

In [7]:
trainDf = trainDf.drop(categoriesCols:_*).na.fill(0)

trainDf = [ID: int, CR_PROD_CNT_IL: int ... 101 more fields]


[ID: int, CR_PROD_CNT_IL: int ... 101 more fields]

In [9]:
val vectorAssembler = new VectorAssembler()
    .setInputCols(trainDf.drop("TARGET").columns)
    .setOutputCol("features")

vectorAssembler = vecAssembler_f91876eabc31


vecAssembler_f91876eabc31

In [11]:
val trainDfCleaned = vectorAssembler.transform(trainDf).cache

trainDfCleaned = [ID: int, CR_PROD_CNT_IL: int ... 102 more fields]


[ID: int, CR_PROD_CNT_IL: int ... 102 more fields]

In [12]:
val gbt = new GBTClassifier()
    .setLabelCol("TARGET")
    .setFeaturesCol("features")
    .setMaxIter(10)
    .setFeatureSubsetStrategy("auto")
    .fit(trainDfCleaned)

gbt = GBTClassificationModel (uid=gbtc_1215114cbc6c) with 10 trees


GBTClassificationModel (uid=gbtc_1215114cbc6c) with 10 trees

In [13]:
import org.apache.spark.ml.linalg.{SparseVector, Vector}
import org.apache.spark.mllib.linalg.{Vector => OldVector}

val vectorToArrayUdf = udf { vec: Any =>
    vec match {
      case v: Vector => v.toArray
      case v: OldVector => v.toArray
      case v => throw new IllegalArgumentException(
        "function vector_to_array requires a non-null input argument and input type must be " +
        "`org.apache.spark.ml.linalg.Vector` or `org.apache.spark.mllib.linalg.Vector`, " +
        s"but got ${ if (v == null) "null" else v.getClass.getName }.")
    }
  }.asNonNullable()

vectorToArrayUdf = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)


import org.apache.spark.mllib.linalg.{Vector=>OldVector}


UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)

# Test model

In [ ]:
val realPredictions = gbt.transform(vectorAssembler.transform(testDf)).cache

In [18]:
val testPath = "/user/sergey.polyakhov/laba05/lab05.csv"

testPath = /user/sergey.polyakhov/laba05/lab05.csv


/user/sergey.polyakhov/laba05/lab05.csv

In [21]:
realPredictions
    .withColumn("target", vectorToArrayUdf(col("probability")).getItem(1))
    .select("ID", "target")
    .coalesce(1)
    .write
    .option("header","true")
    .option("sep" ,"\t")
    .mode("overwrite")
    .format("csv")
    .save("lab05.csv")

In [32]:
spark.stop()